In [1]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from collections import Counter
import yfinance as yf
import glob
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import graphistry
#from graphistry.feature_utils import FeatureMixin, get_dtypes_for_dataframe
import warnings
warnings.filterwarnings('ignore')

In [3]:
# from importlib import reload
# from graphistry import feature_utils
# reload(feature_utils)
# reload(graphistry)
# from graphistry.feature_utils import get_dtypes_for_dataframe

In [4]:
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="", password="") 

In [5]:
g = graphistry.bind() # create base graphistry instance that we can reuse below

# Load in 100 leading industry Tickers

In [6]:
res = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
ndf = res[2]

In [7]:
# load bulk
ndf

,Symbol,Name,Sector
0,AAPL,Apple,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe,Information Technology
...,...,...,...
96,VZ,Verizon,Communication Services
97,WBA,Walgreens Boots Alliance,Consumer Staples
98,WFC,Wells Fargo,Financials
99,WMT,Walmart,Consumer Staples


In [8]:
tickers = ndf[ndf.Symbol != 'BRK.B'].Symbol # since BRK.B is delisted.

# We add in 33 Electric Car Stocks from different parts of the supply chain 
## -- Manufacturers, Battery Systems, Material Stocks, Charging Stations and Equipement

In [9]:
# get Electric Car Stocks + supply chain heuristics
ev_tickers = ['GM', 'F', 'XPEV', 'LI', 'NIO', 'TSLA', 'NKLA', 'RIVN', 'RIDE', 'TM',
              'GOEV', 'FSR', 'APTV', 'FUV', 'BLNK', 'LCID', 'CHPT', 'GELYF', 'MGA', 'DDAIF',
             'PCRFY', 'VWAGY', 'BMWYY', 'ZEV', 'SLDP', 'RMO', 'PCAR', 'LAC', #battery systems
             'NNDM', 'BWA', 'MP', 'FCX', 'ALSN', 'ALB',     #Materials Stocks
             'BEEM', 'BLNK', 'VLTA' # Charging stations + equipment
             ] # add your own here

In [10]:
all_tickers = tickers.to_list()+ev_tickers

In [11]:
if fetch:
    # download their price data
    df_prices_all = yf.download(all_tickers, start='2021-01-01', interval='1h')
    df_prices_all.to_csv('data/stock_prices_all.csv')
else:
    df_prices_all = pd.read_csv('data/stock_prices_all.csv', index_col=0)

In [12]:
df_prices_all

,Adj Close,Adj Close.1,Adj Close.2,Adj Close.3,Adj Close.4,Adj Close.5,Adj Close.6,Adj Close.7,Adj Close.8,Adj Close.9,...,Volume.124,Volume.125,Volume.126,Volume.127,Volume.128,Volume.129,Volume.130,Volume.131,Volume.132,Volume.133
NaN,AAPL,ABBV,ABT,ACN,ADBE,AIG,ALB,ALSN,AMGN,AMT,...,V,VLTA,VWAGY,VZ,WBA,WFC,WMT,XOM,XPEV,ZEV
2021-01-04 09:30:00-05:00,131.05360412597656,105.5,110.0,259.5150146484375,495.8599853515625,37.310001373291016,156.38999938964844,42.95000076293945,227.74000549316406,222.77000427246094,...,1747168.0,NaN,0.0,3788598.0,1792064.0,5918562.0,1901810.0,5976778.0,15792766.0,NaN
2021-01-04 10:30:00-05:00,129.89500427246094,105.01000213623047,108.63999938964844,257.3800048828125,487.9100036621094,37.125,152.80999755859375,42.58000183105469,226.22999572753906,221.10279846191406,...,1111551.0,NaN,63313.0,2124665.0,1095913.0,4288131.0,1867244.0,3916462.0,7798650.0,NaN
2021-01-04 11:30:00-05:00,128.25,104.30999755859375,107.62000274658203,255.51499938964844,480.93499755859375,36.97990036010742,151.36000061035156,42.19499969482422,224.19500732421875,220.75,...,801329.0,NaN,27120.0,1834059.0,1189900.0,3087889.0,1240230.0,2592867.0,5037955.0,NaN
2021-01-04 12:30:00-05:00,129.039794921875,104.48500061035156,108.05000305175781,256.20001220703125,481.29998779296875,37.244998931884766,153.0028076171875,42.189998626708984,224.27999877929688,220.6999969482422,...,623943.0,NaN,15402.0,1269165.0,848710.0,3498474.0,883040.0,1883976.0,2833051.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-11 11:30:00-05:00,171.809998,142.389999,126.629997,334.980011,487.630005,62.505001,236.539993,39.648998,230.404999,239.659607,...,531596.0,349817.0,12168.0,1217919.0,385208.0,2729233.0,618576.0,2414785.0,704315.0,90251.0
2022-02-11 12:30:00-05:00,171.024994,142.460007,126.400002,333.410004,484.220001,61.970001,234.520004,39.490002,230.464996,237.669098,...,705767.0,234469.0,13995.0,1011522.0,424719.0,2439767.0,520580.0,2041832.0,611541.0,60601.0
2022-02-11 13:30:00-05:00,168.979996,142.417999,125.57,329.494995,477.920013,60.759998,230.0,39.080002,229.770004,236.850006,...,1141915.0,400399.0,58298.0,2733935.0,651283.0,5324156.0,1530366.0,7946362.0,1042922.0,132490.0
2022-02-11 14:30:00-05:00,168.835007,142.139999,125.709999,328.924988,477.075012,60.389999,230.889999,39.169998,229.630005,236.610001,...,1058408.0,383183.0,73009.0,3603134.0,1442547.0,4909250.0,1040113.0,4824359.0,1129879.0,177413.0


 This is a multi-Index DataFrame

In [13]:
df_prices_all.columns

Index(['Adj Close', 'Adj Close.1', 'Adj Close.2', 'Adj Close.3', 'Adj Close.4',
       'Adj Close.5', 'Adj Close.6', 'Adj Close.7', 'Adj Close.8',
       'Adj Close.9',
       ...
       'Volume.124', 'Volume.125', 'Volume.126', 'Volume.127', 'Volume.128',
       'Volume.129', 'Volume.130', 'Volume.131', 'Volume.132', 'Volume.133'],
      dtype='object', length=804)

In [14]:
def enrich_from_tickers(tickers):
    """
        Get's enrichment data per ticker
    """
    data = []
    for tick in tickers:
        print(f'Downloading info for {tick}')
        res = yf.Ticker(tick)
        datum = res.info
        data.append(datum)
    return data

In [15]:
if fetch:
    data = enrich_from_tickers(all_tic)
    df  = pd.DataFrame(data)
    df.to_csv('data/stocks_metadata.csv')
else:
    df = pd.read_csv('data/stocks_metadata.csv', index_col=0)
    df['n'] = range(len(df))

In [16]:
# we have many examples of sector
Counter(df.sector).most_common(), df.shape

([('Consumer Cyclical', 31),
  ('Technology', 16),
  ('Healthcare', 16),
  ('Industrials', 16),
  ('Financial Services', 15),
  ('Consumer Defensive', 11),
  ('Communication Services', 10),
  ('Basic Materials', 7),
  ('Utilities', 4),
  ('Energy', 3),
  ('Real Estate', 2),
  (nan, 2)],
 (133, 159))

In [17]:
df

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,website,maxAge,...,preMarketPrice,logo_url,trailingPegRatio,address2,fax,underlyingSymbol,underlyingExchangeSymbol,headSymbol,uuid,n
0,95014,Technology,100000.0,"Apple Inc. designs, manufactures, and markets ...",Cupertino,408 996 1010,CA,United States,https://www.apple.com,1,...,174.00,https://logo.clearbit.com/apple.com,3.3581,NaN,NaN,NaN,NaN,NaN,NaN,0
1,60064-6400,Healthcare,NaN,"AbbVie Inc. discovers, develops, manufactures,...",North Chicago,847 932 7900,IL,United States,https://www.abbvie.com,1,...,143.33,https://logo.clearbit.com/abbvie.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,60064-6400,Healthcare,113000.0,"Abbott Laboratories discovers, develops, manuf...",North Chicago,224 667 6100,IL,United States,https://www.abbott.com,1,...,128.94,https://logo.clearbit.com/abbott.com,2.4397,Abbott Park,NaN,NaN,NaN,NaN,NaN,2
3,2,Technology,624000.0,"Accenture plc, a professional services company...",Dublin,353 1 646 2000,NaN,Ireland,https://www.accenture.com,1,...,346.97,https://logo.clearbit.com/accenture.com,2.7366,Grand Canal Harbour,353 1 646 2020,NaN,NaN,NaN,NaN,3
4,95110-2704,Technology,25988.0,Adobe Inc. operates as a diversified software ...,San Jose,408 536 6000,CA,United States,https://www.adobe.com,1,...,509.14,https://logo.clearbit.com/adobe.com,1.9890,NaN,408 537 6000,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,85004-2189,Basic Materials,NaN,Freeport-McMoRan Inc. engages in the mining of...,Phoenix,602-366-8100,AZ,United States,https://fcx.com,1,...,42.79,https://logo.clearbit.com/fcx.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128
133,46222-3271,Consumer Cyclical,3300.0,"Allison Transmission Holdings, Inc., together ...",Indianapolis,317 242 5000,IN,United States,https://www.allisontransmission.com,1,...,39.00,https://logo.clearbit.com/allisontransmission.com,16.6584,NaN,NaN,NaN,NaN,NaN,NaN,129
134,28209,Basic Materials,5900.0,"Albemarle Corporation develops, manufactures, ...",Charlotte,980-299-5700,NC,United States,https://www.albemarle.com,1,...,234.44,https://logo.clearbit.com/albemarle.com,2.0313,Suite 900,980-299-5512,NaN,NaN,NaN,NaN,130
135,92121,Technology,29.0,"Beam Global, a cleantech company, designs, dev...",San Diego,858 799 4583,CA,United States,https://www.beamforall.com,1,...,14.16,https://logo.clearbit.com/beamforall.com,NaN,NaN,858 799 4592,NaN,NaN,NaN,NaN,131


In [18]:
o = df_prices_all.Open
h = df_prices_all.High
l = df_prices_all.Low
c = df_prices_all.Close

# square of GARMAN-KLASS rv estimator from OLHC data
vol = 1/2 * np.square(np.log(h/l)) - (2*np.log(2)-1)*np.square(np.log(c/o))
vol = vol.replace([np.inf, np.nan], 0)

# square of Rogers-Satchell Volatility 
vol2 = np.log(h/c)*np.log(h/o)+ np.log(l/c)*np.log(l/o)
vol2 = vol2.replace([np.inf, np.nan], 0)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
good_tickers = vol.columns[vol.columns!='BRK.B']

In [ ]:
vol = vol[good_tickers]
vol2 = vol2[good_tickers]

In [ ]:
# have to add in a node identifier, so we use add
vol.T['n'] = range(len(vol.T)) # FIXME this breaks featurizer, why?

# Now we can UMAP Volitility Data

In [ ]:
# Rogers-Satchell Volatility 
g2 = g.nodes(vol2.T)
g3 = g2.umap(scale = 2, n_neighbors=15)

In [ ]:
# g2 = g.nodes(vol.T, 'n').featurize()
# g3 = g2.umap(scale = 2, n_neighbors=15)

In [ ]:
# UMAP has created an implicit Edge DataFrame
g3.weighted_edges_df_from_nodes

In [ ]:
## this will cluster groups according to Rogers-Satchell Volatility
# Story here is a bit confusing, since I don't understand this transform
# But it shows how easy it is to cluster by a given metric of interest!
g4 = g.nodes(df, 'n').edges(g3._edges, '_src', '_dst')
g4.plot()

In [ ]:
#vol.plot()

In [ ]:
#vol2.plot()

# Let's see if we can cluster by  Adj Close value over time. We will see that this naturally clusters by industry category in a semantically useful way. 

In [ ]:
df_prices = df_prices_all['Adj Close']
df_prices = df_prices[good_tickers]

In [ ]:
df_prices

In [ ]:
# we have to run some cleanup on NaN values, so we use Imputer and MinMax Scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

colnames = df_prices.columns
dates = df_prices.index

# impute values 
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df_prices)
res = imputer.transform(df_prices)
df_prices = pd.DataFrame(res, index=df_prices.index, columns = df_prices.columns)

# scale the resulting values
scaler = MinMaxScaler()

df_prices = pd.DataFrame(scaler.fit_transform(df_prices), index=dates, columns=colnames)
df_prices


In [ ]:
df_adj = df_prices.T
df_adj['n'] = range(len(df_adj))

In [ ]:
df_adj

In [ ]:
g2 = g.nodes(df_adj, 'n').featurize()
g3 = g3.umap(scale=2)

In [ ]:
g3.plot()  # LEO why is this happening???

In [ ]:
g4 = g.nodes(df, 'n').edges(g3._edges, '_src', '_dst')
g4.plot()

# Can we use the data from DF (rather than price flows) to cluster by sector? We will see that this does a great job.

In [ ]:
# we want to separate data into Numeric and not, 
gtypes = get_dtypes_for_dataframe(df, verbose=True)

In [ ]:
numeric_df = df[gtypes['float64'] + gtypes['int64']]
numeric_df = numeric_df.replace([np.inf, np.nan], 0)

In [ ]:
numeric_df

In [ ]:
g2 = g.nodes(numeric_df, 'n').featurize()

In [ ]:
#g2 = g.nodes(numeric_df, 'symbol').featurize()
g3 = g2.umap(scale=.5, n_neighbors=10)

In [ ]:
g3.plot() 

In [ ]:
# or seeing it with the better label (company name)
g4 = graphistry.nodes(df, 'n').edges(g3.weighted_edges_df_from_nodes, '_src','_dst')
g4.plot()

# A few amazing observations -- It has clustered, just from financial data, the EV companies by sector. 

It finds all the major EV categories we started with, and even some suprising relationships between correlated stock data. 

## Next we use Textual and Category data and see how well it clusters (sans finance data)

In [ ]:
summary_cols = ['n','longBusinessSummary', 'industry', 'sector', 'financialCurrency', 'longName']

In [ ]:
meta = df[summary_cols]
meta = meta.fillna('')

In [ ]:
g2 = g.nodes(meta, 'n').featurize()

In [ ]:
g3 = g2.umap(scale=0.5, n_neighbors=10)

## We see the similarity from textual and other categorical metadata, and it does quite well

In [ ]:
g3.plot()

# Some stats

In [ ]:
weekly_std = df_prices.resample('W', on='timestamp').std() #.agg(['mean', 'min', 'max', 'std', 'skew'])

In [ ]:
weekly_std.plot(figsize=(17,10))

# Can we run on the volitility data again using a trick to make new features

In [ ]:
df_target = vol

In [ ]:
mdf = df_target.resample('W').agg(['mean', 'min', 'max', 'std', 'skew']).T
mdf

In [ ]:
g2 = g.nodes(mdf)

In [ ]:
g3 = g2.umap()

In [ ]:
g3.weighted_edges_df_from_nodes

In [ ]:
mdf_node_name = np.array(list(df.symbol.values)*5).reshape(5, len(df)).T.flatten()

In [ ]:
hdf = pd.DataFrame({'symbol': list(mdf_node_name)})
hdf['n'] = range(len(mdf))

In [ ]:
g4 = graphistry.nodes(hdf, 'n').edges(g3.weighted_edges_df_from_nodes, '_src','_dst')
g4.plot()

# Lastly Let's compare clustering to a useful PLSR.B regressor, called CCA (Canonical Correlation Analyis)
## We will see that it doesn't do as well as Featurization + UMAP

In [ ]:
from sklearn.cross_decomposition import CCA
clf = CCA(n_components=2)
res=clf.fit_transform(df_prices, df_target)

In [ ]:
res[1].shape

In [ ]:
# two sets of components plotted over time
lf = pd.DataFrame({'comp1_a':res[0].T[0], 
                   'comp2_a':res[0].T[1], 
                   'comp1_b':res[1].T[0], 'comp2_b':res[1].T[1]}, index=df_target.index)
lf.plot(figsize=(15, 10))

In [ ]:
plt.imshow(clf.coef_)

In [ ]:
# we can use the coef_ of clf to make an edge dataframe
a, b = clf.coef_.nonzero()

In [ ]:
CCA_edges = pd.DataFrame({'_src':a, '_dst':b, 'weight': clf.coef_[a,b]})

In [ ]:
from graphistry.feature_utils import prune_weighted_edges_df

In [ ]:
wdf = prune_weighted_edges_df(CCA_edges, scale=3)

In [ ]:
g2 = g.nodes(meta, 'n').edges(wdf, '_src', '_dst')   
g2.plot()  # lots of standalone nodes...